# בס"ד
# Project- Data Mining
# Dudi Achilota

In [19]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm
#from keras.models import Sequential
#from keras.optimizers import Adam

#from yellowbrick.features import ParallelCoordinates
from scipy import stats
import scipy as sc
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'keras'

In [23]:
from IPython.display import Image
Image(url= 'https://cdn.nba.net/nba-drupal-prod/2017-18/SEO-image-NBA-logoman.jpg')

# read excel

In [33]:
#  שחקנים 
df = pd.read_excel('Players.xlsx')
df.sample(3)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
117,Torrey Craig,SF,28,DEN,75,37,1503,160,362,0.442,...,0.700,89,174,263,72,37,46,44,175,430
48,Patrick Beverley,PG,30,LAC,78,49,2137,194,477,0.407,...,0.780,76,312,388,300,67,43,85,265,596
393,Jabari Parker,PF,23,TOT,64,17,1724,369,749,0.493,...,0.712,79,342,421,152,46,30,151,145,930


In [25]:
#שכר
Salary = pd.read_excel('Salary.xlsx')
Salary.sample(3)

,Player,Tm,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
144,Kosta Koufos,SAC,"$8,739,500",NaN,NaN,NaN,NaN,NaN,Cap Space,"$8,739,500"
408,Georges Niang,UTA,"$1,512,601","$1,645,357","$1,783,557",NaN,NaN,NaN,NaN,"$1,512,601"
104,Tyreke Evans,IND,"$12,400,000",NaN,NaN,NaN,NaN,NaN,NaN,"$12,400,000"


In [26]:
TEAM = pd.read_excel('TEAM.xlsx')
TEAM.sample(2)

,TEAM,Tm,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/
4,Houston Rockets,HOU,82,53,29,0.646,48.4,113.9,39.2,87.4,...,31.9,42.1,21.2,13.3,8.5,4.9,4.5,22.0,20.0,4.8
15,Detroit Pistons,DET,82,41,41,0.500,48.4,107.0,38.8,88.3,...,33.6,45.0,22.5,13.8,6.9,4.0,5.1,22.1,21.3,-0.2


# סידור הנתונים

In [27]:
Salary['Guaranteed'] = pd.Series(Salary['Guaranteed']).str.replace(',', '')
Salary['Guaranteed'] = pd.Series(Salary['Guaranteed']).str.replace('$', '')
Salary['2018-19'] = pd.Series(Salary['2018-19']).str.replace(',', '')
Salary['2018-19'] = pd.Series(Salary['2018-19']).str.replace('$', '')

In [28]:
Salary['Guaranteed'] = (Salary['Guaranteed']).astype('int64')
Salary['2018-19'] = (Salary['2018-19']).astype('int64')

Salary = Salary.fillna(0)
Salary.head(2)

,Player,Tm,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
0,Stephen Curry,GSW,37457154,"$40,231,758","$43,006,362","$45,780,966",0,0,Bird Rights,166476240
1,Chris Paul,HOU,35654150,"$38,506,482","$41,358,814","$44,211,146",0,0,0,159730592


In [29]:
Salary['Salary'] = Salary['2018-19']
Salary = Salary[['Player','Salary','Guaranteed']]
df = df.merge(Salary , how='left', on='Player')

In [30]:
df = df[ df.G >= 41]
df = df[ df.Salary >= 10000]

In [ ]:
df.insert(3,'MDD',(df.PTS+ df.TRB+ df.AST+ df.STL+ df.BLK - df.TOV + df.FG - df.FGA)/ df.G)
df.insert(4,'MDD_PER_1MP',(df.PTS+ df.TRB+ df.AST+ df.STL+ df.BLK - df.TOV +df.FG - df.FGA)/ df.MP)
df.insert(5,'PPT3', df['3P'] * 3 /  df['3PA'] )
df.insert(6,'PPT2', df['2P'] * 2 /  df['2PA'] )
df.insert(7,'PPT',(df['3P'] * 3 +df['2P'] * 2 )/ ( df['3PA']+ df['2PA'] ))
df.insert(8,'PtsToMP',(df.PTS)/ df.MP)

In [ ]:
df = df.drop(columns=['GS','ORB','DRB'])
df['PTS'] = df['PTS'] / df['G'] 
df['PF']  = df['PF'] / df['G'] 

df['TOV'] = df['TOV'] / df['G']
df['MP']  = df['MP'] /df['G']

df['BLK'] = df['BLK'] / df['G']
df['STL'] = df['STL'] / df['G'] 
df['AST'] = df['AST'] / df['G'] 
df['TRB'] = df['TRB'] / df['G']

In [ ]:
df['3PA'] = df['3PA'] / df['G']
df['2PA'] = df['2PA'] / df['G']
df['FGA'] = df['FGA'] / df['G']    
df['FTA'] = df['FTA'] / df['G'] 

df['FT'] = df['FT'] / df['G']    
df['3P'] = df['3P'] / df['G']    
df['2P'] = df['2P'] / df['G']    
df['FG'] = df['FG'] / df['G'] 

In [ ]:
df['ThrowFavorite'] = np.where(df['PPT3'] > df['PPT2'],3 , 2)

In [ ]:
df['Salary'] = df['Salary'] / 1000000

In [ ]:
df = df.fillna(0)

In [ ]:
df[(df['Pos'] == 'C-PF') |(df['Pos'] == 'PF-SF') |(df['Pos'] == 'SF-SG') |(df['Pos'] == 'SG-PF')|(df['Pos'] == 'SG-SF')]
df['Pos'] = df['Pos'].replace('SG-PF', 'SF')
df['Pos'] = df['Pos'].replace('SG-SF', 'SG')
df['Pos'] = df['Pos'].replace('SF-SG', 'SG')
df['Pos'] = df['Pos'].replace('PF-SF', 'SF')
df['Pos'] = df['Pos'].replace('C-PF', 'PF')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 3 to 576
Data columns (total 31 columns):
Player        365 non-null object
Pos           365 non-null object
Age           365 non-null int64
Tm            365 non-null object
G             365 non-null int64
GS            365 non-null int64
MP            365 non-null int64
FG            365 non-null int64
FGA           365 non-null int64
FG%           365 non-null float64
3P            365 non-null int64
3PA           365 non-null int64
3P%           351 non-null float64
2P            365 non-null int64
2PA           365 non-null int64
2P%           365 non-null float64
eFG%          365 non-null float64
FT            365 non-null int64
FTA           365 non-null int64
FT%           365 non-null float64
ORB           365 non-null int64
DRB           365 non-null int64
TRB           365 non-null int64
AST           365 non-null int64
STL           365 non-null int64
BLK           365 non-null int64
TOV           365 non-nu

In [32]:
df.isnull().sum().sum()

14

# 1.1   הצגת נתונים

In [ ]:
df.describe()

In [ ]:
f,ax= plt.subplots(figsize=(8,5))
plt.hist(df['Age'],np.arange(19,40))
plt.grid(True)
plt.title("Histogram")
plt.xlabel("Age")

f,ax= plt.subplots(figsize=(8,5))
df['Pos'].value_counts().plot.pie(subplots=True, figsize=(12, 6))
plt.grid(True)

f,ax= plt.subplots(figsize=(8,5))
plt.hist(df['PTS'],np.arange(0,40))
plt.grid(True)
plt.title("Histogram")
plt.xlabel("PTS")

f,ax= plt.subplots(figsize=(8,5))
plt.hist(df['MDD'],np.arange(0,40))
plt.grid(True)
plt.title("Histogram")
plt.xlabel("MDD")
plt.ylabel("")

f,ax= plt.subplots(figsize=(8,5))
plt.hist(df['Salary'],np.arange(0,40))
plt.grid(True)
plt.title("Histogram")
plt.xlabel("Salary")

In [ ]:
Q1 = df[['Player','PPT3','PPT2','3P%','3P','3PA','2P%','2P','2PA','G']]
Q1 = Q1[ (Q1['3PA'] >= 2) & (Q1['3P'] >= 0.9)]
Q1 = Q1[ (Q1['2PA'] >= 4) & (Q1['2P'] >= 2)]

In [ ]:
Q1.describe()

In [ ]:
Q1['ThrowFavorite'] = np.where(Q1['PPT3'] > Q1['PPT2'],3 , 2)
Q1['PercentFromFavorite'] = np.where(Q1['PPT3'] > Q1['PPT2'],Q1['PPT3'] , Q1['PPT2'])
Q1.sort_values('PercentFromFavorite', ascending=False).head()

In [ ]:
Q1['ThrowFavorite'].value_counts().plot(kind='bar')

In [ ]:
Q1 = Q1.merge(df[['Pos','Player','Tm']] , how='left', on='Player')

In [ ]:
Q1['Pos'].value_counts().plot.bar()

In [ ]:
f,ax= plt.subplots(figsize=(8,5))
Q1[Q1.ThrowFavorite==2]['Pos'].value_counts().plot.bar(color='brown')
plt.grid(True)

f,ax= plt.subplots(figsize=(8,5))
Q1[Q1.ThrowFavorite==3]['Pos'].value_counts().plot.bar(color='red')
plt.grid(True)

In [ ]:
df['POSֹ'] = df['Pos']
df['POSֹ'] = df['POSֹ'].replace('PG', 1)
df['POSֹ'] = df['POSֹ'].replace('SG' ,2)
df['POSֹ'] = df['POSֹ'].replace('SF', 3)
df['POSֹ'] = df['POSֹ'].replace('PF', 4)
df['POSֹ'] = df['POSֹ'].replace('C',  5)

# 1.2

In [ ]:
df.corr()
sns.set()
df_corr = df._get_numeric_data()  #אתה יכול להשתמש בפונקציה הלא מתועד _get_numeric_data()לסינון עמודות מספריות בלבד:

mask = np.zeros_like(df_corr.corr(),dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap =  sns.palplot(sns.diverging_palette(150, 275, s=80, l=55, n=9))

plt.figure(figsize=(50,20))
sns.heatmap(df_corr.corr(),cmap=cmap,annot=True,mask = mask,square = True)
#plt.show()

In [ ]:
#ax = sns.clustermap(df.corr() , standard_scale = 1  )

In [ ]:
TEAM_corr = TEAM._get_numeric_data()  #אתה יכול להשתמש בפונקציה הלא מתועד _get_numeric_data()לסינון עמודות מספריות בלבד:
TEAM_corr = TEAM_corr.drop(['GP','W','L','MIN'],axis=1)

mask = np.zeros_like(TEAM_corr.corr(),dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap =  sns.palplot(sns.diverging_palette(150, 275, s=80, l=55, n=9))

plt.figure(figsize=(50,20))
sns.heatmap(TEAM_corr.corr(),cmap=cmap,annot=True,mask = mask,square = True)
#plt.show()

In [ ]:
#ax = sns.clustermap(TEAM_corr.corr() , standard_scale = 1  )

# 2.1
# Player Ratio Group  רגרסיה לינארית

In [ ]:
PlayerRatioGroup = df[['Player','Tm','G','Age','Salary','MP','PTS','AST','TRB']].merge(TEAM[['Tm','PTS','MIN','AST','REB']]
                                                                              ,   how='left', on='Tm')

In [ ]:
PlayerRatioGroup.isnull().sum()

In [ ]:
PlayerRatioGroup= PlayerRatioGroup.dropna()

In [ ]:
PlayerRatioGroup['PTS'] = PlayerRatioGroup['PTS_x'] / PlayerRatioGroup['PTS_y'] *100   
PlayerRatioGroup['AST'] = PlayerRatioGroup['AST_x'] / PlayerRatioGroup['AST_y']  *100  
PlayerRatioGroup['RB'] = PlayerRatioGroup['TRB'] / PlayerRatioGroup['REB']    *100

In [ ]:
PlayerRatioGroup = PlayerRatioGroup[ PlayerRatioGroup.Salary > .5]
PlayerRatioGroup.sample(1)

In [ ]:
PlayerRatioGroup.describe()

In [ ]:
bins = [0, 23 ,26, 30, 43]
category = [ 1 ,2 ,3 ,4 ]
PlayerRatioGroup['AgeCategory'] = pd.cut(PlayerRatioGroup['Age'], bins, labels = category)

In [ ]:
PlayerRatioGroup['PTS_TOGAME'] = PlayerRatioGroup['PTS_x']

In [ ]:
PlayerRatioGroup = PlayerRatioGroup[['Player','G',"PTS", "AST", 'RB',"Salary",'PTS_TOGAME','AgeCategory']]
PlayerRatioGroup['SUM'] = PlayerRatioGroup.PTS + PlayerRatioGroup.AST + PlayerRatioGroup.RB

PlayerRatioGroup.sort_values('PTS', ascending=False).head()

In [ ]:
scatter_matrix(PlayerRatioGroup[["PTS", "AST", 'RB',"Salary"]], alpha = 0.5, figsize=(10, 6))

In [ ]:
PlayerRatioGroup[ PlayerRatioGroup.PTS_TOGAME >= 10].corr()["Salary"].sort_values()

In [ ]:
R1 = PlayerRatioGroup.sort_values('SUM', ascending=False).head(70)
R2 = PlayerRatioGroup.sort_values('Salary', ascending=False).head(70)
R3 = R1.merge(R2, on='Player')
R3[['Player']].head()
R3.shape

In [ ]:
PlayerRatioGroup[ PlayerRatioGroup.G >= 53].corr()["Salary"].sort_values()

In [ ]:
PlayerRatioGroup = PlayerRatioGroup[ PlayerRatioGroup.G >= 53]

In [ ]:
PRG_corr = PlayerRatioGroup._get_numeric_data() 

mask = np.zeros_like(PRG_corr.corr(),dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap =  sns.palplot(sns.diverging_palette(150, 275, s=80, l=55, n=9))

plt.figure(figsize=(20,10))
sns.heatmap(PRG_corr.corr(),cmap=cmap,annot=True,mask = mask,square = True)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.scatter(x=PlayerRatioGroup['AST'],y=PlayerRatioGroup['Salary'],c='w',label='AST')
#plt.scatter(x=PlayerRatioGroup['SUM'],y=PlayerRatioGroup['Salary'],c='k',label='SUM')
plt.scatter(x=PlayerRatioGroup['PTS'],y=PlayerRatioGroup['Salary'],c='b',label='PTS')
plt.scatter(x=PlayerRatioGroup['RB'],y=PlayerRatioGroup['Salary'],c='g',label='RB')

plt.plot(PlayerRatioGroup['AST'], sm.ols(formula="Salary ~ AST", data=PlayerRatioGroup).fit().predict(),c='w',linewidth=3)
plt.plot(PlayerRatioGroup['PTS'], sm.ols(formula="Salary ~ PTS", data=PlayerRatioGroup).fit().predict(),c='b',linewidth=3)
plt.plot(PlayerRatioGroup['RB'],  sm.ols(formula="Salary ~ RB", data=PlayerRatioGroup).fit().predict(),c='g',linewidth=3)

plt.legend(numpoints=1,loc=4)
plt.ylabel('Salary')
plt.show()

In [ ]:
sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().summary()

In [ ]:
PlayerRatioGroup[['SUM','Salary']].corr()["Salary"].sort_values()

In [ ]:
plt.scatter(x=PlayerRatioGroup['SUM'],y=PlayerRatioGroup['Salary'],c='r',label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),c='b',linewidth=7)

plt.legend(numpoints=1,loc=4)
plt.ylabel('Salary')
plt.show()

# [0, 23 ,26, 30, 43]

In [ ]:
PlayerRatioGroup['AgeCategory'].value_counts().plot.bar()

In [ ]:
f,ax= plt.subplots(figsize=(10,5))# 3 כתום# 1 שחור

plt.scatter(x=PlayerRatioGroup['SUM'],y=PlayerRatioGroup['Salary'], c=PlayerRatioGroup['AgeCategory']  ,label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),linewidth=7)
plt.grid(True)
plt.legend(numpoints=1,loc=4)
plt.title("TO AGE")
plt.ylabel('Salary')
plt.xlabel('Dominant and significant to the group')
plt.show()
# bins = [0, 23 ,26, 30, 43]

f,ax= plt.subplots(figsize=(10,5))
plt.scatter(x=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==1]['SUM']
        ,y=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==1]['Salary'] ,c='r'    ,label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),linewidth=7)

plt.title("TO AGE 19 - 23")
plt.ylabel('Salary')
plt.xlabel('Dominant and significant to the group')
plt.grid(True)
plt.show()

f,ax= plt.subplots(figsize=(10,5))

plt.scatter(x=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==2]['SUM']
        ,y=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==2]['Salary']  ,c='g'   ,label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),linewidth=7)
plt.title("TO AGE 23 - 26")
plt.ylabel('Salary')
plt.xlabel('Dominant and significant to the group')
plt.grid(True)
plt.show()

f,ax= plt.subplots(figsize=(10,5))

plt.scatter(x=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==3]['SUM']
        ,y=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==3]['Salary'] ,c='r'    ,label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),linewidth=7)
plt.title("TO AGE 26 - 30")
plt.ylabel('Salary')
plt.xlabel('Dominant and significant to the group')
plt.grid(True)
plt.show()

f,ax= plt.subplots(figsize=(10,5))

plt.scatter(x=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==4]['SUM']
        ,y=PlayerRatioGroup[PlayerRatioGroup.AgeCategory ==4]['Salary'] , c='g'     ,label='SUM')
plt.plot(PlayerRatioGroup['SUM'], sm.ols(formula="Salary ~ SUM", data=PlayerRatioGroup).fit().predict(),linewidth=7)
plt.title("TO AGE 30 - 43")
plt.ylabel('Salary')
plt.xlabel('Dominant and significant to the group')
plt.grid(True)
plt.show()

#  2.2  מכונה 
#  SUM SALARY PlayerRatioGroup  

In [ ]:
X = PlayerRatioGroup[['SUM']].values
y = PlayerRatioGroup['Salary'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
model = Sequential()

model.add(Dense(1, input_shape=(1,)))

model.add(Activation('linear'))
model.summary()

In [ ]:
model.compile(Adam(lr = 50), loss='mean_squared_error')

In [ ]:
es = EarlyStopping(monitor='val_loss',min_delta=0, patience=10,verbose=2, mode='auto')

In [ ]:
model.fit(X_train,  y_train,  epochs= 900,
          validation_data=(X_test,y_test), callbacks=[es])

In [ ]:
W, B = model.get_weights()

print('The slope of the regression line: %.2f' % (W))#שיפוע
print('The intercept for the regression line: %.2f' % (B))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(PlayerRatioGroup['SUM'], PlayerRatioGroup['Salary'], 'o')
plt.ylabel('Salary')
plt.xlabel('Ratio Group SUM')
plt.title('Player Ratio Group SUM vs Salary')

plt.plot(X_test , y_pred, linewidth=5)

In [ ]:
test_score = r2_score(y_test, y_pred)
print('R-squared for the test set: %.2f' % (test_score))

In [ ]:
plt.plot(PlayerRatioGroup['SUM'] , PlayerRatioGroup['Salary'], 'o')
plt.ylabel('Salary')
plt.title('Player Ratio Group PTS vs Salary')
plt.plot(X_test,y_pred,color='black',linewidth=3 )

In [ ]:
#הערך בריבוע R נמוך יותר עבור ערכת הבדיקה מאשר עבור מערך הרכבות (0.39 <0.45) כך שנראה שהמודל מתאים לסט הרכבת
y_pred_train = model.predict(X_train)

train_score = r2_score(y_train, y_pred_train)
print('R-squared for the train set: %.2f' % (train_score))

In [ ]:
y_pred_all = model.predict(X)

score = r2_score(y, y_pred_all)
print('R-squared for all of the data: %.2f' % (score))

In [ ]:
y_test_f32 = y_test.astype('float32').reshape(56)
y_pred_f32 = y_pred.astype('float32').reshape(56)
corr, p_value = pearsonr(y_pred_f32 , y_test_f32)
print('P-value for the test set: ' + str(p_value))

# 2.3
# חיזוי שכר באמצעות למידת מכונה ומודל לינארי מרובה משתנים

In [ ]:
PlayerRatioGroup.corr()['Salary'].sort_values()

In [ ]:
PlayerRatioGroup[ PlayerRatioGroup.Salary > 5].corr()["Salary"].sort_values()

In [ ]:
plt.plot(PlayerRatioGroup['PTS_TOGAME'], PlayerRatioGroup['Salary'], 'o')
plt.ylabel('Salary')

In [ ]:
plt.plot(PlayerRatioGroup['PTS'], PlayerRatioGroup['Salary'], 'o')
plt.ylabel('Salary')

In [ ]:
plt.plot(PlayerRatioGroup['AST'], PlayerRatioGroup['Salary'], 'o')
plt.ylabel('Salary')

In [ ]:
X = PlayerRatioGroup[['PTS','SUM','PTS_TOGAME']]
y = PlayerRatioGroup['Salary']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
model = Sequential()

model.add(Dense(1, input_shape=(3,)))

model.add(Activation('linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=20), 'mean_squared_error')

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0, patience=10,
                   verbose=2, mode='auto')

In [ ]:
model.fit(X_train, y_train,
          batch_size=32,  epochs=1500,
          validation_data=(X_test,y_test), callbacks=[es])

In [ ]:
(W1, W2, W3), B = model.get_weights()
X_test.head(3)

In [ ]:
X_test.loc[0]

In [ ]:
y_test.loc[0]

In [ ]:
y_pred7 = 12.096070 * W1 + 38.470521 * W2 + 15.689300 * W3 + 38.470521* W3 + B
y_pred7[0]

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test.values.reshape(74), y_pred.reshape(74))

idx = ['slope', 'intercept', 'r_value', 'p_value', 'std_err']
data = np.array([slope, intercept, r_value, p_value, std_err])
fd = pd.DataFrame(data = data, index = idx , columns = ['values'])
fd
#ערך r של  אומר לנו שהמודל מצליח להסביר 67% מהשונות בתוצאות.
#ערך p שואף לאפס, מלמד שהקשר אותו מתאר המודל הוא מובהק ביותר, ואינו מקרי.

In [ ]:
# משוואת הישר 
predict_line = lambda S: 4.309718e-01 * y_test + 4.056186e+00

In [ ]:
two_pts_for_the_line = np.array([y_test.min(), y_test.max()])

predicted_line = predict_line(two_pts_for_the_line)

In [ ]:
plt.plot(y_test, y_pred, 'bo')
plt.plot(y_test, predicted_line, c='RED')
plt.xlabel("")

plt.tight_layout()

In [ ]:
#מידת הסטייה בין התחזית למחירים בפועל
print('MAE: %.2f' % metrics.mean_absolute_error(y_test, y_pred))
print('MSE: %.2f' % metrics.mean_squared_error(y_test, y_pred))
print('RMSE: %.2f'% np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#אומר שגיאה מוחלטת
#משמעותם של מדדים היא שגיאה בריבוע

In [ ]:
model1 = LinearRegression()

model1.fit(X_train,y_train)

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
#חישוב רגרסיה לינארית פחות ריבועית לשתי קבוצות של מדידות.
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test, y_pred)

idx = ['slope', 'intercept', 'r_value', 'p_value', 'std_err']
data = np.array([slope, intercept, r_value, p_value, std_err])

pd.DataFrame(data = data, index = idx , columns = ['values'])

In [ ]:
predict_line = lambda S: 4.796025e-01 * y_test + 4.530588e+00

two_pts_for_the_line = np.array([y_test.min(), y_test.max()])

predicted_line = predict_line(two_pts_for_the_line)

In [ ]:
plt.plot(y_test, y_pred, 'bo')
plt.plot(y_test, predicted_line, c='RED')
plt.xlabel("")

plt.tight_layout()

# 3.1
# Linear Regression

# "MDD", "PTS", "Salary"

In [ ]:
scatter_matrix(df[["MDD", "PTS", "Salary"]], alpha = 1, figsize=(13, 6 ))

In [ ]:
df.corr()["Salary"].sort_values().tail(8)

In [ ]:
df.corr()["MDD"].sort_values().tail(15)

In [ ]:
plt.scatter(x=df['MDD'],y=df['Salary'],c='b',marker='o',label='MDD')
plt.scatter(x=df['PTS'],y=df['Salary'],c='k',marker='*',label='PTS')

plt.legend(numpoints=1,loc=4)
plt.xlabel('MDD || PTS')
plt.ylabel('Salary')
plt.show()

In [ ]:
ols1 = sm.ols(formula="Salary ~ MDD", data=df).fit()
ols1.summary()

In [ ]:
ols2 = sm.ols(formula="Salary ~ PTS", data=df).fit()
ols2.summary()

In [ ]:
plt.scatter(x=df['MDD'],y=df['Salary'],c='b',marker='o',label='MDD')
plt.scatter(x=df['PTS'],y=df['Salary'],c='k',marker='*',label='PTS')
plt.legend(numpoints=1,loc=4)


plt.plot(df['MDD'],ols1.predict(),c='b',linewidth=3)
plt.plot(df['PTS'],ols2.predict(),c='k',linewidth=3)

plt.xlabel('MDD')
plt.ylabel('Salary')
plt.show()

# 3.2
# Multiple Linear Regression

In [ ]:
ols3 = sm.ols(formula="Salary ~ MDD +  PTS", data=df).fit()
ols3.summary()

In [ ]:
ols4 = sm.ols(formula="Salary ~ MDD +  PTS + MDD * PTS", data=df).fit()
ols4.summary()

# 3.3  MDD

In [ ]:
plt.scatter(x=df['AST'],y=df['MDD'],c='r',marker='s',label='AST')
plt.scatter(x=df['TRB'],y=df['MDD'],c='b',marker='o',label='TRB')
plt.scatter(x=df['PTS'],y=df['MDD'],c='k',marker='*',label='PTS')

plt.legend(numpoints=1,loc=4)
plt.xlabel('AST || TRB || PTS')
plt.ylabel('MDD ')
plt.show()

In [ ]:
ols = sm.ols(formula="MDD ~ PTS", data=df).fit()
ols.summary()

In [ ]:
plt.scatter(x=df['PTS'],y=df['MDD'],c='k',marker='*',label='PTS')
plt.plot(df['PTS'], ols.predict(),'k',color='blue',linewidth=2)

plt.xlabel('PTS')
plt.ylabel('MDD')
plt.show()

In [ ]:
ols2 = sm.ols(formula="MDD ~ TRB", data=df).fit()
ols2.summary()

In [ ]:
plt.scatter(x=df['TRB'],y=df['MDD'],c='k',marker='*',label='TRB')
plt.plot(df['TRB'], ols2.predict(),'k',color='blue',linewidth=1.5)

plt.xlabel('TRB')
plt.ylabel('MDD')
plt.show()

In [ ]:
ols1 = sm.ols(formula="MDD ~ AST ", data=df).fit()
ols2 = sm.ols(formula="MDD ~ TRB ", data=df).fit()
ols3 = sm.ols(formula="MDD ~ PTS ", data=df).fit()

plt.scatter(x=df['AST'],y=df['MDD'],c='r',marker='s',label='AST')
plt.scatter(x=df['TRB'],y=df['MDD'],c='b',marker='o',label='TRB')
plt.scatter(x=df['PTS'],y=df['MDD'],c='k',marker='*',label='PTS')
plt.legend(numpoints=1,loc=4)

plt.plot(df['AST'], ols1.predict(),c='r',linewidth=2)
plt.plot(df['TRB'], ols2.predict(),c='b',linewidth=2)
plt.plot(df['PTS'], ols3.predict(),c='k',linewidth=2)

plt.xlabel('')
plt.ylabel(' ')
plt.show()


In [ ]:
all_ols = sm.ols(formula="MDD ~ PTS + TRB ", data=df).fit()
all_ols.summary()

# 4.1
# TEAM ~~~ playoffs

In [ ]:
TEAM = TEAM.drop(columns=['Tm','W','L','MIN','GP'])

In [ ]:
TEAM.describe()

In [ ]:
TEAM['playoffs'] = np.where(TEAM['WIN%'] >= 0.5, 1 , 0)

In [ ]:
TEAM.sample(2)

In [ ]:
TEAM.plot.scatter(x='P3P',y='playoffs',s=55)

In [ ]:
model1 = sm.logit(formula=" playoffs ~ P3P ", data=TEAM).fit()
model1.summary()

In [ ]:
x = sc.linspace(-10,10,1000)
logistic = lambda x: 1/(1+sc.exp(-x))

TEAM.plot.scatter(x='P3P',y='playoffs',s=30)
x = sc.linspace(TEAM['P3P'].min(),TEAM['P3P'].max(),1000)
par = dict( model1.params)
plt.plot(x,logistic(par['Intercept'] + par['P3P']*x),color='Red')
plt.show()

In [ ]:
logistic(par['Intercept'] + par['P3P'] * .35)

In [ ]:
model2 = sm.logit(formula=" playoffs ~ FGF ", data=TEAM).fit()
model2.summary()

In [ ]:
#logistic(par['Intercept'] + par['FGF'] * 0.04)
model2.predict(pd.DataFrame({"FGF": [0.004]}))

In [ ]:
TEAM.plot.scatter(x='FGF',y='playoffs',s=30)
x = sc.linspace(TEAM['FGF'].min(),TEAM['FGF'].max(),1000)
par = dict( model2.params)
plt.plot(x,logistic(par['Intercept'] + par['FGF']*x),color='Red')
plt.show()

In [ ]:
TEAM['T_F'] = model1.predict()> 0.55
pd.crosstab(index=TEAM["playoffs"], columns=TEAM["T_F"])

In [ ]:
print(23/30)

print(19/30)

In [ ]:
TEAM['T_F'] = model1.predict()> 0.7
pd.crosstab(index=TEAM["playoffs"], columns=TEAM["T_F"])

In [ ]:
TEAM.plot.scatter(x='PTS',y='playoffs',s=40)

In [ ]:
sm.logit(formula=" playoffs ~ PTS ", data=TEAM).fit().summary()

# 5.1
# POS הבחנה בין קבוצות עמדות

In [ ]:
#1
plt.figure(figsize=(7,7))
sns.set(style='ticks')
sns.pairplot(df[['Pos','FT%','2P%','3P%','eFG%']],hue = 'Pos' )

In [ ]:
P1 = df[['Pos','FT%','2P%','3P%','eFG%']]
x = P1[['FT%','2P%','3P%','eFG%']].values
y = P1[['Pos']].values

In [ ]:
POS_names = np.unique(np.array(y))
POS_names

In [ ]:
names_dict = {k: v for v, k in enumerate(POS_names)}
names_dict

In [ ]:
s = pd.DataFrame(y)
y_cat = pd.get_dummies(s)
y_cat.sample(2)

In [ ]:
X = pd.DataFrame(x)
X_train, X_test, y_train, y_test = train_test_split(X.values, y_cat, test_size=0.25,random_state=42)

In [ ]:
model = Sequential()

model.add(Dense(5,input_shape=(4,),activation='softmax'))
model.compile(Adam(lr=0.1),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss',min_delta=0.001,patience=5,verbose=2,mode='auto')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test),callbacks=[es],epochs=1200)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
y_actual = np.argmax(np.array(y_test),axis=1)
y_actual

In [ ]:
print(classification_report(y_actual, y_pred,target_names = POS_names))
#דיויק
# 

In [ ]:
cm = confusion_matrix(POS_names[y_actual], POS_names[y_pred],POS_names)

In [ ]:
plt.figure(figsize=(10,9))
print(names_dict)
sns.heatmap(cm,annot=True,xticklabels=True,yticklabels=True)
plt.xlabel('Actual_ לעופב')
plt.ylabel('Predicted_ יוזח')


In [ ]:
def cm_analysis(y_true, y_pred, filename, labels, ymap=None, figsize=(10,10)):
      
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual_ לעופב'
    cm.columns.name = 'Predicted_ יוזח'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    plt.savefig(filename)

In [ ]:
#{0:'C', 1:'PF',2:'PG' , 3: 'SF' , 4: 'SG' }
cm_analysis(y_actual, y_pred, 'figur1',[2,4,3,1,0],  ymap={0:'C', 1:'PF',2:'PG' , 3: 'SF' , 4: 'SG' })
POS_names

# 5.2
# פחות משתנים

In [ ]:
#2
P2 = df[['Pos','FT%','2P%','3P%','eFG%']]
P2['Pos'] = P2['Pos'].replace('C', 'C-PF')
P2['Pos'] = P2['Pos'].replace('PF', 'C-PF')

P2['Pos'] = P2['Pos'].replace('SG', 'PG-SG')
P2['Pos'] = P2['Pos'].replace('PG', 'PG-SG')

sns.set(style='ticks')
sns.pairplot( P2 , hue = 'Pos'  )

In [ ]:
#3
P2['Pos'] = P2['Pos'].replace('SF', 'PG-SG')
sns.set(style='ticks')
sns.pairplot( P2 , hue = 'Pos'  )

In [ ]:
x = P2[['FT%','2P%','3P%','eFG%']].values
y = P2[['Pos']].values

In [ ]:
POS_names = np.unique(np.array(y))
POS_names

In [ ]:
names_dict = {k: v for v, k in enumerate(POS_names)}
names_dict

In [ ]:
s = pd.DataFrame(y)
y_cat = pd.get_dummies(s)
y_cat.sample(2)

In [ ]:
X = pd.DataFrame(x)

X_train, X_test, y_train, y_test = train_test_split(X.values, y_cat, test_size=0.25,random_state=42)

In [ ]:
model = Sequential()

model.add(Dense(2,input_shape=(4,),activation='softmax'))
model.compile(Adam(lr=0.1),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss',min_delta=0.001,patience=5,verbose=1,mode='auto')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test),callbacks=[es],epochs=1200)

In [ ]:
y_pred = model.predict(X_test)
y_pred[:5]

In [ ]:
# קבל מדדי המחלקה שקיבלו את ההסתברות הגבוהה ביותר עבור כל מדגם
y_pred = np.argmax(y_pred,axis=1)
y_pred

In [ ]:
y_actual = np.argmax(np.array(y_test),axis=1)
y_actual

# נעשה רשימה של שמות  שקיבלנו

In [ ]:
print(classification_report(y_actual, y_pred,target_names = POS_names))

In [ ]:
# cm = confusion_matrix(y_actual,y_pred)
cm = confusion_matrix(POS_names[y_actual], POS_names[y_pred],POS_names)
cm

In [ ]:
def cm_analysis(y_true, y_pred, filename, labels, ymap=None, figsize=(10,10)):
      
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual_ לעופב'
    cm.columns.name = 'Predicted_ יוזח'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    plt.savefig(filename)

In [ ]:
plt.figure(figsize=(10,9))
sns.heatmap(cm,annot=True,xticklabels=True,yticklabels=True)
plt.xlabel('Actual_ לעופב')
plt.ylabel('Predicted_ יוזח')

In [ ]:
cm_analysis(y_actual, y_pred, 'figur1', [0,1], ymap={0:'C-PF' , 1: 'PG-SG' })
POS_names

# 6.1
# עץ החלטה

In [ ]:
TeamsLastYears = pd.read_excel('C:/TeamsLastYears.xlsx')
TeamsLastYears.sample(2)

In [ ]:
TeamsLastYears['WIN%'].describe()

In [ ]:
TeamsLastYears['NOsuccess'] = np.where(TeamsLastYears['WIN%'] >  0.59, 0 , 1)
TeamsLastYears['success'] = np.where(TeamsLastYears['WIN%'] >= 0.59, 1 , 0)

In [ ]:
features = ["PTS", "AST","REB", "+/", "P3P",  "FGF", "TOV"]
classes = ["success","NOsuccess" ]

X = TeamsLastYears[features]
y = TeamsLastYears["success"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
def splitData(features):
    """Split a subset of the titanic dataset, given by the features, into train and test sets."""
    predictors = TeamsLastYears[features].values
    labels = TeamsLastYears["success"].values

    XTrain, XTest, yTrain, yTest = train_test_split( predictors, labels, random_state=1, test_size=0.5)
    return XTrain, XTest, yTrain, yTest

In [ ]:
from IPython.display import Image, display  
import pydotplus 
from scipy import misc

def renderTree(my_tree, features):
    filename = "temp.dot"
    with open(filename, 'w') as f:
        f = tree.export_graphviz(my_tree, out_file=f, 
 feature_names=features,  class_names=["success","NOsuccess" ],   filled=True, rounded=True,special_characters=True)
    dot_data = ""
    with open(filename, 'r') as f:
        dot_data = f.read()

    graph = pydotplus.graph_from_dot_data(dot_data)
    image_name = "temp.png"
    graph.write_png(image_name)  
    display(Image(filename=image_name))

In [ ]:
decisionTree = tree.DecisionTreeClassifier()

XTrain, XTest, yTrain, yTest = splitData(["+/"])
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data = ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data = ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

renderTree(decisionTree, ["+/"])

In [ ]:
all_features = ["PTS", "AST","REB", "+/", "P3P",  "FGF", "TOV"]

XTrain, XTest, yTrain, yTest = splitData(all_features)
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

In [ ]:
features = [ "AST","REB",  "P3P",  "FGF", "TOV"]
classes = ["success","NOsuccess" ]

X = TeamsLastYears[features]
y = TeamsLastYears["success"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
all_features = [ "AST","REB",  "P3P",  "FGF", "TOV"]


XTrain, XTest, yTrain, yTest = splitData(all_features)
decisionTree = tree.DecisionTreeClassifier(max_depth=4)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

In [ ]:
features = [  "P3P",  "FGF", 'FT%']
classes = ["success","NOsuccess" ]

X = TeamsLastYears[features]
y = TeamsLastYears["success"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
all_features = [  "P3P",  "FGF", 'FT%']


XTrain, XTest, yTrain, yTest = splitData(all_features)
decisionTree = tree.DecisionTreeClassifier(max_depth=3)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

# 6.2
# playoffs

In [ ]:
features = ["PTS", "+/", "P3P", "TOV", "FGF", "PF", "AST"]
classes = ["playoffs","NO" ]

X = TEAM[features]
y = TEAM["playoffs"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
all_features =  ["PTS", "+/", "P3P", "TOV", "FGF", "PF", "AST"]

XTrain, XTest, yTrain, yTest = splitData(all_features)
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

# 6.3

In [ ]:
df['MDD'].describe()

In [ ]:
df['Stars'] = np.where(  df['MDD'] > 18, 1 , 0)

#df['NOStars'] = np.where(  df['MDD'] >  18, 0 , 1)

In [ ]:
df['Stars'].value_counts().plot.bar()

In [ ]:
features = ['3P%','2P%' ,'FT%' ,'eFG%' ]
classes = ["Stars","NOStars" ]

X = df[features]
y = df["Stars"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
def splitData2(features):
    """Split a subset of the titanic dataset, given by the features, into train and test sets."""
    predictors = df[features].values
    labels = df["Stars"].values

    XTrain, XTest, yTrain, yTest = train_test_split( predictors, labels, random_state=1, test_size=0.5)
    return XTrain, XTest, yTrain, yTest

In [ ]:
def renderTree2(my_tree, features):
    filename = "temp.dot"
    with open(filename, 'w') as f:
        f = tree.export_graphviz(my_tree, out_file=f, 
 feature_names=features,  class_names=["Stars","NOStars" ],   filled=True, rounded=True,special_characters=True)
    dot_data = ""
    with open(filename, 'r') as f:
        dot_data = f.read()

    graph = pydotplus.graph_from_dot_data(dot_data)
    image_name = "temp.png"
    graph.write_png(image_name)  
    display(Image(filename=image_name))

In [ ]:
all_features = ['3P%','2P%' ,'FT%' ,'eFG%' ]

XTrain, XTest, yTrain, yTest = splitData2(all_features)
decisionTree = tree.DecisionTreeClassifier(max_depth=3)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree2(decisionTree, all_features)

In [ ]:
features = ['PF' ,'STL' ,'FTA' ]
classes = ["Stars","NOStars" ]

X = df[features]
y = df["Stars"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()

In [ ]:
XTrain, XTest, yTrain, yTest = splitData2(features)
decisionTree = tree.DecisionTreeClassifier(max_depth=3)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree2(decisionTree, features)

In [ ]:
features = ['TOV','Age' ]
classes = ["Stars","NOStars" ]

X = df[features]
y = df["Stars"]

visualizer = ParallelCoordinates(classes=classes, features=features, sample=0.5, shuffle=True,  normalize='minmax')

visualizer.fit_transform(X, y)

visualizer.poof()
XTrain, XTest, yTrain, yTest = splitData2(features)
decisionTree = tree.DecisionTreeClassifier(max_depth=2)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree2(decisionTree, features)

# סיום